In [1]:
import torch
import os
import shutil
import torchvision as tv
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToPILImage
from torch import nn as nn
import numpy as np
import time
show = ToPILImage()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
path = 'E:\InteliJ Programes\Pycharm\dataset\kaggle\kagglecatsanddogs\PetImages'
labels = os.listdir(path)
print(labels)

['Cat', 'Dog']


In [3]:
cat_path = os.path.join(path, 'Cat')
dog_path = os.path.join(path, 'Dog')
cats_vs_dogs_dataset_path = 'E:\InteliJ Programes\Pycharm\dataset\kaggle\kagglecatsanddogs\dataset'
train_path = os.path.join(cats_vs_dogs_dataset_path, 'train')
valid_path = os.path.join(cats_vs_dogs_dataset_path, 'valid')
test_path = os.path.join(cats_vs_dogs_dataset_path, 'test')
print(train_path)

E:\InteliJ Programes\Pycharm\dataset\kaggle\kagglecatsanddogs\dataset\train


In [4]:
def make_dirs(): # 生成文件夹
    if not os.path.isdir(cats_vs_dogs_dataset_path):
        print('当前路径下不存在文件夹，现在创建文件夹')
        os.mkdir(cats_vs_dogs_dataset_path)
        os.mkdir(train_path)
        os.mkdir(valid_path)
        os.mkdir(test_path)
        for i in enumerate(labels):
            os.mkdir(os.path.join(train_path, i[1]))
            os.mkdir(os.path.join(valid_path, i[1]))
            os.mkdir(os.path.join(test_path, i[1]))
        print("文件夹创建完成")
    else:
        print("当前文件夹已存在")
        
def check_dirs(): #检查文件夹是否都存在
    print(f'{train_path}:{os.path.isdir(train_path)}')

In [5]:
make_dirs()

当前文件夹已存在


In [6]:
# 数据集文件复制到train、valid、test文件夹下

def load_train_images(n=7500):
    fnames = ['{}.jpg'.format(i) for i in range(n)]
    for i in enumerate(labels):
        for fname in fnames:
            src = os.path.join(os.path.join(path, i[1]), fname)
            dst = os.path.join(os.path.join(train_path, i[1]), fname)
            shutil.copyfile(src, dst)
            
def load_valid_images():
    fnames = ['{}.jpg'.format(i) for i in range(7500,10625)]
    for i in enumerate(labels):
        for fname in fnames:
            src = os.path.join(os.path.join(path, i[1]), fname)
            dst = os.path.join(os.path.join(valid_path, i[1]), fname)
            shutil.copyfile(src, dst)
            
def load_test_images():
    fnames = ['{}.jpg'.format(i) for i in range(10625,12450)]
    for i in enumerate(labels):
        for fname in fnames:
            src = os.path.join(os.path.join(path, i[1]), fname)
            dst = os.path.join(os.path.join(test_path, i[1]), fname)
            shutil.copyfile(src, dst)

In [7]:
load_train_images()
load_valid_images()
load_test_images()

In [8]:
train_tsfm = tt.Compose([
                    tt.ToTensor(),
                    tt.Resize((128, 128)),
                    tt.RandomHorizontalFlip(),
                    #tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                    ])
valid_test_tsfm = tt.Compose([
                    tt.ToTensor(),
                    tt.Resize((128, 128)),
                    #tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                    ])

In [9]:
train_dataset = ImageFolder(
                        root=train_path,
                        transform=train_tsfm)
valid_dataset = ImageFolder(
                        root=valid_path,
                        transform=valid_test_tsfm)
test_dataset = ImageFolder(
                        root=test_path,
                        transform=valid_test_tsfm)

train_loader = torch.utils.data.DataLoader(
                    dataset=train_dataset,
                    batch_size=32,
                    shuffle=True,
                    num_workers=2
                    )
valid_loader = torch.utils.data.DataLoader(
                    dataset=valid_dataset,
                    batch_size=32,
                    shuffle=False,
                    num_workers=2
                    )
test_loader = torch.utils.data.DataLoader(
                    dataset=test_dataset,
                    batch_size=32,
                    shuffle=False,
                    num_workers=2
                    )

In [10]:
data, label = test_dataset[3649]
#print(len(train_dataset))
print(labels[label])
show((data))
print(data)

Dog
tensor([[[0.3213, 0.3951, 0.2514,  ..., 0.3826, 0.2750, 0.3223],
         [0.3945, 0.4150, 0.2623,  ..., 0.2011, 0.6644, 0.3462],
         [0.4337, 0.4123, 0.2221,  ..., 0.2194, 0.6747, 0.5077],
         ...,
         [0.6922, 0.6888, 0.7027,  ..., 0.7061, 0.5732, 0.5970],
         [0.7294, 0.6981, 0.6955,  ..., 0.6727, 0.6353, 0.5497],
         [0.6906, 0.6978, 0.7003,  ..., 0.6675, 0.7273, 0.5174]],

        [[0.3566, 0.3987, 0.2436,  ..., 0.4228, 0.3231, 0.3751],
         [0.4411, 0.4253, 0.2554,  ..., 0.2311, 0.7063, 0.4065],
         [0.4915, 0.4351, 0.2182,  ..., 0.2317, 0.7125, 0.5704],
         ...,
         [0.6897, 0.6796, 0.6846,  ..., 0.7368, 0.6050, 0.6323],
         [0.7254, 0.6825, 0.6759,  ..., 0.7001, 0.6510, 0.5854],
         [0.6867, 0.6821, 0.6807,  ..., 0.6949, 0.7430, 0.5567]],

        [[0.2864, 0.3163, 0.1612,  ..., 0.4161, 0.3517, 0.3923],
         [0.3762, 0.3473, 0.1731,  ..., 0.1813, 0.7245, 0.4120],
         [0.4438, 0.3624, 0.1358,  ..., 0.1139, 0.7089

In [11]:
# dataiter = iter(train_loader)
# image, label = dataiter.next() # 返回8张图片及标签，因为之前封装的方法batch_size=8
# image, label = image.to(device), label.to(device)
# print(''.join('%11s'%labels[label[j]] for j in range(8)))
# show(tv.utils.make_grid(image))

In [18]:
# mat1 and mat2 shapes cannot be multiplied (64x8192 and 2048x1024) (32x32768 and 2048x1024)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
           # nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64] 
 
            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64] 
           # nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
          #  nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
          #  nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
           # nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
           # nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128), 
            nn.ReLU(), 
            nn.Linear(128,2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [19]:
model = Classifier()
model.to(device)

Classifier(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   

In [20]:
learning_rate = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [30]:
epochs = 2
for epoch in range(epochs):
    epoch_start_time = time.time()
    train_acc =0.0
    val_acc =0.0
    train_loss = 0.0
    val_loss =0.0
    model.train()
    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        image, label = data[0].to(device), data[1].to(device)
        output = model(image)
        
        loss = criterion(output, label.long())
        loss.backward()
        optimizer.step()
        
        train_acc += np.sum(np.argmax(output.cpu().data.numpy(),axis=1)== label.cpu().numpy())
        train_loss +=loss.item()
    model.eval()
    with torch.no_grad():
        for i,data in enumerate(valid_loader):
            valx ,valy = data[0].to(device),data[1].to(device)
            val_pred = model(valx)
            batch_loss = criterion(val_pred,valy.long())
            val_acc +=np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1)== valy.cpu().numpy())
            val_loss +=batch_loss.item()

        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, epochs, time.time()-epoch_start_time, \
             train_acc/train_dataset.__len__(), train_loss/train_dataset.__len__(), val_acc/valid_dataset.__len__(), val_loss/valid_dataset.__len__()))

[001/002] 52.09 sec(s) Train Acc: 0.779533 Loss: 0.014438 | Val Acc: 0.809440 loss: 0.013229
[002/002] 50.38 sec(s) Train Acc: 0.825133 Loss: 0.012021 | Val Acc: 0.834080 loss: 0.011399
